In [ ]:
!pip install flask requests transformers

In [ ]:
%%writefile app.py
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)

# Initialize sentiment analysis pipeline with distilbert
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Initialize text generation pipeline with google/flan-t5-large.
# Note: T5 is a text-to-text model so we use the "text2text-generation" pipeline.

def check_sentiment(message):
    result = sentiment_pipeline(message)
    return result[0]  # e.g., {'label': 'NEGATIVE', 'score': 0.99}

@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    user_message = data.get("message", "").strip()

    # Check sentiment using the distilbert model
    sentiment_result = check_sentiment(user_message)

    # If negative sentiment is detected, return a safe response.
    if sentiment_result['label'] == "NEGATIVE":
        return jsonify({
            "reply": "I sense some anger in your words. Let's try to keep the conversation calm.",
            "sentiment_analysis": sentiment_result
        })

    # Build a prompt that includes the conversation history.
    # For T5, we provide a text-to-text prompt.
    prompt = """
    """

    # Generate a response using google/flan-t5-large
    full_text = None

    return jsonify({
        "reply": full_text,
        "sentiment_analysis": sentiment_result,
    })

if __name__ == '__main__':
    # The server will run on 0.0.0.0:5000
    app.run(host='0.0.0.0', port=5000)


In [ ]:
!nohup python app.py &


In [ ]:
import socket
import time

tries = 0

def print_last_lines(filename, num_lines=10):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            total_lines = len(lines)
            start_index = max(0, total_lines - num_lines)
            for line in lines[start_index:]:
                print(line, end="")
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")

def check_port(host, port):
  port_closed = True
  time.sleep(1)
  tries = 0
  while port_closed and tries < 15:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        try:
            sock.connect((host, port))
            print(f"Port {port} on {host} is open!")
            port_closed = False
        except ConnectionRefusedError:
            print(f"Port {port} on {host} is closed.")
            print_last_lines('nohup.out')
            tries += 1
            time.sleep(5)


check_port("127.0.0.1", 5000)

In [ ]:
import requests

url = "http://127.0.0.1:5000/chat"

# Test messages
messages = [

]

for msg in messages:
    payload = {"message": msg}
    response = requests.post(url, json=payload)
    if response.ok:
        result = response.json()
        print(f"User: {msg}")
        print("Chatbot:", result.get("reply"))
        print("Sentiment Analysis:", result.get("sentiment_analysis"))
        if "conversation_history" in result:
            print("Conversation History:")
            for line in result["conversation_history"]:
                print("   " + line)
        print("-" * 50)
    else:
        print("Failed to connect to the chatbot service.")
